In [1]:
from parser import *
from snapshotExtraction import extract_snapshot, get_most_recent_workflows
import pandas as pd
import os
from ruamel.yaml import YAML

## Extracting information from a single workflow file


In [5]:
df = pd.read_csv('../dataset/200_workflowsonly.csv')
print(df.shape[0])

firstWorkflow = df.iloc[0]
file_hash = firstWorkflow['file_hash']

folder_path = f"../dataset/workflows"
file_path = os.path.join(folder_path, file_hash)

2595399


In [6]:
if not os.path.isfile(file_path):
    print(f"File {file_path} not found.")
else:
    parsed_data = parse_workflow(file_path)

    if parsed_data:
        print(f"Workflow analysé : {parsed_data['file_path']}")
        print(f"- Nombre de lignes : {parsed_data['lines_count']}")
        print(f"- Déclencheurs : {parsed_data['events']}")
        print(f"- Nombre de jobs : {parsed_data['jobs_count']}")

        for job, details in parsed_data["jobs"].items():
            print(f"  * Job: {job}")
            print(f"    - Nombre de steps: {details['steps_count']}")
            print(f"    - Utilise GitHub Actions: {details['uses_github_actions']}")
            print(f"    - Utilise des commandes shell: {details['uses_commands']}")
            print(f"    - Détails des steps :")
            for step in details["step_details"]:
                print(f"      - {step['name']}: uses={step['uses']}, run={step['run']}")

Workflow analysé : ../dataset/workflows\6f1b3e145aeb96e714212eb0b5e05088d7d6b50a4a945a3488779a94a85c80f0
- Nombre de lignes : 27
- Déclencheurs : ['push', 'pull_request']
- Nombre de jobs : 1
  * Job: build
    - Nombre de steps: 6
    - Utilise GitHub Actions: True
    - Utilise des commandes shell: True
    - Détails des steps :
      - Checkout code: uses=actions/checkout@v3, run=None
      - Setup node: uses=actions/setup-node@v3, run=None
      - Install dependencies: uses=None, run=if ! yarn install; then
  cat /tmp/xfs-*/buildfile.log 2>/dev/null || true
  exit 1
fi

      - Lint: uses=None, run=yarn lint
      - Check types: uses=None, run=yarn types:check
      - Build: uses=None, run=yarn build:chrome


## Take a snapshot of the dataset
- Filter the dataset to keep only the repositories that were committed before the snapshot date with a limit of 1 month before the snapshot date.
- For each uid in the snapshot, get the most recent workflow.

In [12]:
snapshot = extract_snapshot(df, 2024)
most_recent_workflows = get_most_recent_workflows(snapshot, 2024)
print(f"Nombre de workflows dans le snapshot : {snapshot.shape[0]}")
print(f"Nombre de workflows les plus récents : {most_recent_workflows.shape[0]}")

Nombre de workflows dans le snapshot : 67013
Nombre de workflows les plus récents : 31230


C:\Users\Thomas\PycharmProjects\memoire\customcode\snapshotExtraction.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snapshot['days_from_date'] = (snapshot['committed_date'] - snapshot_date).abs()
